<a href="https://colab.research.google.com/github/Muideen-Abubakar/Dissertation/blob/main/Dissertation_Code_pt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **Preparing Notebook Environment**

###### **Installing libraries**

In [ ]:
%%capture
!conda install pytorch torchvision -c pytorch
!conda install numpy pandas matplotlib regex
!conda install -c conda-forge scikit-learn
!conda install -c conda-forge nltk
!conda install -c conda-forge openpyxl
!pip install openpyxl
!pip install PyPDF2 pdfplumber
!pip install pytest-shutil
!pip install transformers

###### **Importing libraries**

In [ ]:
%%capture
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import regex as re
import pdfplumber
import os, sys #to combine pdfs
import nltk #set of libraries for Natural Language Processing
from sklearn.feature_extraction.text import TfidfVectorizer #tf-idf formula
from difflib import SequenceMatcher #flexible class for comparing pairs of sequences
import torch
import os
from glob import glob
import nltk.data
import shutil
from transformers import AutoTokenizer, AutoModelForSequenceClassification

#### **Preparing Data**

###### **Pulling Cost of Capital for each Industry**

---



In [ ]:
url = 'https://github.com/Muideen-Abubakar/Dissertation/blob/main/Cost_of_Capital.xlsx?raw=true'

energy_wacc = pd.read_excel(url, sheet_name='Energy')
agric_wacc = pd.read_excel(url, sheet_name='Agric_and_Cons_Staple')
industrial_wacc = pd.read_excel(url, sheet_name='Industrial')
real_estate_wacc = pd.read_excel(url, sheet_name='R_Estate')
transportation_wacc = pd.read_excel(url, sheet_name='Transportation')

In [ ]:
print("Energy Cost of Capital\n")
print(energy_wacc.head(3))

print("\nAgriculture Cost of Capital\n")
print(agric_wacc.head(3))

print("\nIndustrial Cost of Capital\n")
print(industrial_wacc.head(3))

Energy Cost of Capital

  Ticker             Company    2017    2018    2019    2020    2021
0    CVX        Chevron Corp  0.0694  0.0900  0.0727  0.0792  0.1004
1    NEE  NextEra Energy Inc  0.0441  0.0583  0.0452  0.0636  0.0794
2    COP      ConocoPhillips  0.0902  0.0858  0.0687  0.0735  0.0935

Agriculture Cost of Capital

  Ticker                     Company    2017    2018    2019    2020    2021
0   MDLZ  Mondelez International Inc  0.0815  0.0752  0.0640  0.0601  0.0707
1    ADM   Archer-Daniels-Midland Co  0.0751  0.0778  0.0618  0.0615  0.0764
2    ECL                  Ecolab Inc  0.0811  0.0896  0.0697  0.0879  0.1020

Industrial Cost of Capital

  Ticker          Company    2017    2018    2019    2020    2021
0    LIN        Linde PLC  0.0512  0.0990  0.0868  0.0956  0.1450
1    CAT  Caterpillar Inc  0.0807  0.0992  0.0783  0.0605  0.0709
2    MMM            3M Co  0.0820  0.1109  0.0816  0.0656  0.0745


In [ ]:
print("Real Estate Cost of Capital\n")
print(real_estate_wacc.head(3))

print("\nTransportation Cost of Capital\n")
print(transportation_wacc.head(3))

Real Estate Cost of Capital

  Ticker              Company    2017    2018    2019    2020    2021
0    AMT  American Tower Corp  0.0696  0.0726  0.0520  0.0606  0.0727
1    PLD         Prologis Inc  0.0708  0.0764  0.0645  0.0821  0.1010
2   EQIX          Equinix Inc  0.0602  0.0690  0.0538  0.0654  0.0779

Transportation Cost of Capital

  Ticker                    Company    2017    2018    2019    2020    2021
0   TSLA                  Tesla Inc  0.1270  0.0856  0.0692  0.1239  0.1513
1    UNP         Union Pacific Corp  0.0772  0.0938  0.0732  0.0767  0.0892
2    UPS  United Parcel Service Inc  0.0705  0.0986  0.0782  0.0642  0.0769


###### **Pulling ESG Reports**

***The reports or data retrieved here pertains to the Energy/Electric Power Sector. The relevant variables/parameters should be changed accordingly for each industry being processed***

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%capture
%%R
# Signify cell is an R cell
# Silence Output

h = install.packages("pacman")
h = library(pacman)
p_load("tidyverse", "googledrive")

# killing auth request
drive_deauth()
drive_user()

# retrieving fulll url ID of files
public_file = drive_get(as_id(c("1WYoGl-44QqeuTW-cK9srkLkNDnkOoj7v")) ) # G.Drive ID of the zipped folder of sustainability reports. 

# download the files
save_output = 1 %>% map(~drive_download(public_file[.,], overwrite = T))

In [ ]:
%%capture
!unzip SAData.zip #change folder name accordingly for each sector being processed
!unzip Data.zip

###### **Preprocessing Sample Energy Documents**

In [ ]:
# Preprocessing
nltk.download('punkt')
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

def extractDoc(doc = None):
    if os.path.exists(doc):
        pdf = pdfplumber.open(doc)
        n = len(pdf.pages)
        final = ""
        for page in range(n):
            data = pdf.pages[page].extract_text()
            final = final + "\n" + data
        return final


def normalizeText(text = None):
    """
    """
    text = text.encode("utf-8", "ignore").decode("utf-8")
    text = text.replace("\n"," ")
    text = ' '.join(text.split())
    return text
  

def cleanText(text = None, sent_detector = None):
    """   
    """
    textList = sent_detector.tokenize(text.strip())
    return textList

def getSentences(filepath = None, sent_detector = sent_detector):
    text = extractDoc(filepath)
    text = normalizeText(text)
    text = cleanText(text, sent_detector)
    return(text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
companyNames = ('Chevron', 'Nextera','Conoco', 'Duke', 'Southern', 'EOG', 'Pioneer', 'Dominion', 'Schlumberger') # for each folder, create this tuple of complany names
companyIDs = ('CVX', 'NEE', 'COP', 'DUK', 'SO', 'EOG', 'PXD', 'D', 'SLB') # modify the tickers to match the company names accordingly
years = ('2017', '2018', '2019', '2020', '2021')

def makeData(sent_detector = sent_detector, companyNames = None, companyIDs = None, years = None):

    pdfsInFolder = glob("Data/NYSE*.pdf", recursive=True)

    print("{} PDF files were matched".format(len(pdfsInFolder)))

    processed, Data  = 0, [[], [], []]

    for pos, pdfFilepath in enumerate(pdfsInFolder):
        try:
            sentenceList = getSentences(pdfFilepath, sent_detector)
            filePathSplitList = pdfFilepath.split("_")
            fileYear = [filePathSplitList[-1].split('.')[0] for i in range(len(sentenceList))]
            fileCompany = [companyNames[companyIDs.index(filePathSplitList[1])] for i in range(len(sentenceList))]
            processed +=1
            Data[0].extend(fileCompany)
            Data[1].extend(fileYear)
            Data[2].extend(sentenceList)
            if ((pos+1) % 3  == 0 or pos == len(pdfsInFolder) - 1):
                print("Processed {0} out of {1} PDF files. File id: {2}".format(processed, len(pdfsInFolder), pdfFilepath))
        except:
            print("Something went wrong while processing file: {}".format(pdfFilepath))
    sentenceDf = pd.DataFrame({"Company": Data[0], "Year":Data[1], "Sentence": Data[2]})            
    print("\n\n\nFinished processing all PDF files")
    return(sentenceDf)

In [ ]:
cleanData = makeData(sent_detector = sent_detector, companyNames = companyNames, companyIDs = companyIDs, years = years)

21 PDF files were matched
Processed 3 out of 21 PDF files. File id: Data/NYSE_NEE_2021.pdf
Processed 6 out of 21 PDF files. File id: Data/NYSE_SO_2016.pdf
Processed 9 out of 21 PDF files. File id: Data/NYSE_CVX_2018.pdf
Processed 12 out of 21 PDF files. File id: Data/NYSE_NEE_2017.pdf
Processed 15 out of 21 PDF files. File id: Data/NYSE_CVX_2019.pdf
Processed 18 out of 21 PDF files. File id: Data/NYSE_SO_2018.pdf
Processed 21 out of 21 PDF files. File id: Data/NYSE_SO_2017.pdf



Finished processing all PDF files


In [ ]:
display(cleanData['Company'].value_counts())
print("")
display(cleanData.head(3))

Conoco      13393
Southern     8054
Chevron      3623
Nextera      2282
Name: Company, dtype: int64

,Company,Year,Sentence
0,Nextera,2018,2018 EDISON ELECTRIC INSTITUTE ESG/SUSTAINABIL...
1,Nextera,2018,Our vision is to be North America’s leader in ...
2,Nextera,2018,"Edison Electric Institute (EEI), along with co..."


In [ ]:
# This is where you save the sentence file
np.savetxt(r'Sentences.txt', cleanData['Sentence'].values, encoding='utf-8', fmt='%s')

#### **Preparing Model**


###### **Cloning FinBERT Model**

In [ ]:
if not os.path.exists("finbert"):
    # !git clone https://huggingface.co/ProsusAI/finbert
    !git clone https://github.com/ProsusAI/finBERT.git
else:
    print("Git folder already pulled")


Cloning into 'finBERT'...
remote: Enumerating objects: 168, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 168 (delta 6), reused 10 (delta 4), pack-reused 149
Receiving objects: 100% (168/168), 73.57 KiB | 1.01 MiB/s, done.
Resolving deltas: 100% (67/67), done.


In [ ]:
!cp -R finBERT/. /content/
!rm -rf finBERT/

#### **Performing Inference**

In [ ]:
from finbert.finbert import predict
from transformers import AutoModelForSequenceClassification
import argparse
import os

In [ ]:
text_path = 'Sentences.txt'
output_dir = 'output/'
model_path = 'ProsusAI/finbert'

if not os.path.exists(output_dir):
    os.mkdir(output_dir)


with open(text_path,'r') as f:
    text = f.read()

model = AutoModelForSequenceClassification.from_pretrained(model_path,num_labels=3,cache_dir=None)

output = "predictions.csv"
predict(text, model, write_to_csv=True, path=os.path.join(output_dir,output))

Streaming output truncated to the last 5000 lines.
08/06/2022 04:49:20 - INFO - finbert.utils -   input_ids: 101 3461 19279 2226 8817 1997 2329 9829 3197 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
08/06/2022 04:49:20 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
08/06/2022 04:49:20 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
08/06/2022 04:49:20 - INFO - finbert.utils -   label: None (id = 9090)
08/06/2022 04:49:21 - INFO - root -   tensor([[-0.8320, -1.1648,  2.5380],
        [-0.6743, -1.1357,  2.2128],
        [-0.6280, -0.8680,  1.4529],
        [-0.8639, -0.8781,  2.2595],
        [-0.6280, -0.8680,  1.4529]])
08/06/2022 04:49:21 - INFO - finbert.utils -   *** Exa

,sentence,logit,prediction,sentiment_score
0,2018 EDISON ELECTRIC INSTITUTE ESG/SUSTAINABIL...,"[0.3716884, 0.007959446, 0.62035215]",neutral,0.363729
1,Our vision is to be North America’s leader in ...,"[0.84755886, 0.006787652, 0.1456535]",positive,0.840771
2,"Edison Electric Institute (EEI), along with co...","[0.20996444, 0.007424218, 0.78261137]",neutral,0.202540
3,This template is divided into qualitative and ...,"[0.024426254, 0.02807267, 0.94750106]",neutral,-0.003646
4,The qualitative section focuses on sustainabil...,"[0.05745447, 0.013625031, 0.92892045]",neutral,0.043829
...,...,...,...,...
27329,"Southern resources, as well as changes in appl...","[0.020828066, 0.042955924, 0.93621606]",neutral,-0.022128
27330,The reader is cautioned not timing of the entr...,"[0.09333759, 0.022695163, 0.8839673]",neutral,0.070642
27331,Southern Company and be realized.,"[0.08433364, 0.009317193, 0.9063492]",neutral,0.075016
27332,"The following factors, in addition to deployme...","[0.023030998, 0.029020462, 0.9479485]",neutral,-0.005989


#### **Combining Results**

In [ ]:
from glob import glob
import pandas as pd
import os

###### **Pulling data and post-inference output**

In [ ]:
%%capture
%load_ext rpy2.ipython

In [ ]:
%%capture

%%R

h = install.packages("pacman")
h = library(pacman)
p_load("tidyverse", "googledrive")

drive_deauth()
drive_user()

public_file = drive_get(as_id(c("1Uz2dvFtMW1Ndz7u56TBFauEgXEmWgipm")) )

save_output = 1 %>% map(~drive_download(public_file[.,], overwrite = T))

***Note that due to file size and to optimize run time speed, each sector data and consequently, output files were split into two. Making a total of 10 files.*** 

In [ ]:
!unzip finbert_data_and_output.zip

Archive:  finbert_data_and_output.zip
  inflating: finbert_data/Sentences_Agric1.tsv  
  inflating: finbert_data/Sentences_Agric2.tsv  
  inflating: finbert_data/Sentences_Energy1.tsv  
  inflating: finbert_data/Sentences_Energy2.tsv  
  inflating: finbert_data/Sentences_Industrials1.tsv  
  inflating: finbert_data/Sentences_Industrials2.tsv  
  inflating: finbert_data/Sentences_Realestate1.tsv  
  inflating: finbert_data/Sentences_Realestate2.tsv  
  inflating: finbert_data/Sentences_Transport1.tsv  
  inflating: finbert_data/Sentences_Transport2.tsv  
  inflating: finbert_output/Agric1.csv  
  inflating: finbert_output/Agric2.csv  
  inflating: finbert_output/Energy1.csv  
  inflating: finbert_output/Energy2.csv  
  inflating: finbert_output/Industrials1.csv  
  inflating: finbert_output/Industrials2.csv  
  inflating: finbert_output/Realestate1.csv  
  inflating: finbert_output/Realestate2.csv  
  inflating: finbert_output/Transport1.csv  
  inflating: finbert_output/Transport2.csv 

In [ ]:
output_files = glob(os.path.join("finbert_output", "*.csv"), recursive=True)
print("Output files matched: ".format(output_files))
data_files = glob(os.path.join("finbert_data", "*.tsv"), recursive=True)
print("Data files matched: ".format(data_files))
result = pd.DataFrame(columns=['Company', 'Year', 'Sentence', 'logit', 'prediction', 'sentiment_score'])
for file in output_files:
  name = file.split("/")[-1]
  name = name.split(".")[0]
  print("Processing: {} files".format(name))
  data_file = [ matched_file for matched_file in data_files if name in matched_file][0]
  data_df = pd.read_csv(data_file, sep = "\t").reset_index()
  output_df = pd.read_csv(file).reset_index()
  batch_result = pd.merge(data_df, output_df, left_on= "Sentence", right_on= "sentence")
  batch_result = batch_result[['Company', 'Year', 'Sentence', 'logit', 'prediction', 'sentiment_score']]
  result = pd.concat([result, batch_result], ignore_index=True)
result.to_csv("Inference_result.tsv", sep="\t", quotechar="'")
print("Completed merging of {} files: ".format(len(data_files)))
print("Result saved in Inference_result.tsv")

Output files matched: 
Data files matched: 
Processing: Energy1 files
Processing: Transport1 files
Processing: Agric2 files
Processing: Transport2 files
Processing: Realestate2 files
Processing: Realestate1 files
Processing: Agric1 files
Processing: Industrials1 files
Processing: Energy2 files
Processing: Industrials2 files
Completed merging of 10 files: 
Result saved in Inference_result.tsv


###### **Download inference result**

In [ ]:
from google.colab import files
files.download("Inference_result.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>